In [74]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision.transforms as transforms
import torchvision.models as models

from PIL import Image
import matplotlib.pyplot as plt

import copy

from pytorch_datasetloader import *

# The Discriminator

In [75]:
# class Dis(nn.Module):
#     def __init__(self):
        
#         super().__init__()
        
#         self.model = nn.Sequential(

#             #block 1 input = 3*192*192
#             nn.Conv2d(in_channels=3,out_channels=15,kernel_size=3),
#             # 15*190*190
#             nn.ReLU(),
#             nn.MaxPool2d(3, stride=2),
#             # 15*95*95
            
            
#             #block 2 input = 15*95*95
#             nn.Conv2d(in_channels=15,out_channels=50,kernel_size=3),
#             # 50*93*93
#             nn.ReLU(),
#             nn.MaxPool2d(3, stride=2),
#             # 50*46*46
            
            
#             #block 3 input = 50*46*46
#             nn.Conv2d(in_channels=50,out_channels=200,kernel_size=3),
#             # 200*44*44
#             nn.ReLU(),
#             nn.MaxPool2d(3, stride=2),
#             # 200*22*22
            
#             nn.reshape(out.size(0), -1)
            
#             #Fully Connected
#             nn.Linear(in_features=200*22*22,out_features=120),
#             nn.Linear(in_features=120,out_features=60),
#             nn.Linear(in_features=60,out_features=1),
#             nn.Sigmoid()
            
            
            
            
#         )
    
#     def forward(self, input):
#         return self.model(input)



In [76]:
class Dis(nn.Module):
    
    #Our batch shape for input x is (3, 32, 32)
    
    def __init__(self):
        
        super().__init__()
        
        #block 1 input = 3*128*128 
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=3,out_channels=15, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        
        #block 2 input = 15*64*64
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=15,out_channels=50, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        
        #block 3 input = 50*32*32
        self.layer3 = nn.Sequential(
            nn.Conv2d(in_channels=50,out_channels=200, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        
     
        #self.drop_out = nn.Dropout()
        self.fc1 = torch.nn.Linear(200*16*16, 120)
        self.fc2 = torch.nn.Linear(120, 60)
        
        self.outlayer = nn.Sequential(
            nn.Linear(60, 1),
            nn.Sigmoid()
        )
        
        
        
    def forward(self, x):
        

        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.reshape(out.size(0), -1)
        #out = self.drop_out(out)
        out = self.fc1(out)
        out = self.fc2(out)
        out = self.outlayer(out)
        
        
        return(out)
    


In [77]:
#using cuda if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

D = Dis()
D = D.to(device)


cuda:0


# Loading Dataset

In [78]:
path = 'images/'
trainloader, validloader = get_loaders(path)


769 images from the dataset


# Selecting Optimizer 

In [79]:
criterion = nn.BCELoss()
d_optimizer = torch.optim.Adam(D.parameters(), lr=0.0002)
#g_optimizer = torch.optim.Adam(G.parameters(), lr=0.0003)

# Pretraining Discriminator Seperately

In [105]:
# Train the model
total_step = len(trainloader)
loss_list = []
acc_list = []
num_epochs = 100
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(trainloader):
        
        images = images.to(device) 
        #labels = labels.to(device)
#         labels_compare = labels
#         labels_compare = labels.to(device)
        labels = torch.tensor(labels, dtype=torch.float, device=device)
        
#         continue
        
        # Run the forward pass
        outputs = D(images)

        loss = criterion(outputs, labels)
        loss_list.append(loss.item())

        # Backprop and perform Adam optimisation
        d_optimizer.zero_grad()
        loss.backward()
        d_optimizer.step()

        # Track the accuracy
        total = labels_compare.size(0)
        
        preds = (outputs >= 0.7)
        preds = preds.reshape(-1)
        preds = torch.tensor(preds, dtype=torch.float, device=device)
        #print(preds)
        

#         , predicted = torch.max(outputs.data, 1)
        
        correct = (preds == labels).sum().item()
        acc_list.append(correct / total)

        if (i + 1) % 10 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                  .format(epoch + 1, num_epochs, i + 1, total_step, loss.item(),
                          (correct / total) * 100))

Epoch [1/100], Step [10/17], Loss: 0.2569, Accuracy: 87.50%
Epoch [2/100], Step [10/17], Loss: 0.2543, Accuracy: 87.50%
Epoch [3/100], Step [10/17], Loss: 0.2500, Accuracy: 87.50%
Epoch [4/100], Step [10/17], Loss: 0.2446, Accuracy: 87.50%
Epoch [5/100], Step [10/17], Loss: 0.2378, Accuracy: 87.50%
Epoch [6/100], Step [10/17], Loss: 0.2302, Accuracy: 87.50%
Epoch [7/100], Step [10/17], Loss: 0.2217, Accuracy: 87.50%
Epoch [8/100], Step [10/17], Loss: 0.2136, Accuracy: 87.50%
Epoch [9/100], Step [10/17], Loss: 0.2065, Accuracy: 87.50%
Epoch [10/100], Step [10/17], Loss: 0.1975, Accuracy: 87.50%
Epoch [11/100], Step [10/17], Loss: 0.1894, Accuracy: 87.50%
Epoch [12/100], Step [10/17], Loss: 0.1812, Accuracy: 87.50%
Epoch [13/100], Step [10/17], Loss: 0.1729, Accuracy: 87.50%
Epoch [14/100], Step [10/17], Loss: 0.1638, Accuracy: 87.50%
Epoch [15/100], Step [10/17], Loss: 0.1553, Accuracy: 87.50%
Epoch [16/100], Step [10/17], Loss: 0.1474, Accuracy: 87.50%
Epoch [17/100], Step [10/17], Los

KeyboardInterrupt: 